In [ ]:
# CNN model for predicting Sex from ECGs
import tensorflow as tf # Make sure that python interpreter is 3.9.13 Global env
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import wfdb
import ast
import ecg_plot
tf.config.list_physical_devices('GPU')

### Loading raw data into mutable Datframes
ptb = pd.read_csv('../data/ptbxl_database.csv')
ptb.head()
len(ptb)
def load_raw_data(df, sampling_rate, path):
    data = [wfdb.rdsamp(path+f) for f in df.filename_lr]
    data = np.array([signal for signal, meta in data])
    return data
sampling_rate=100
# load and convert annotation data
Y = pd.read_csv('../data/ptbxl_database.csv', index_col='ecg_id')
Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))


# Load raw signal data
X = load_raw_data(Y, sampling_rate, '../data/')
ecg_plot.plot_12(tf.transpose(X, (0,2,1))[0], sample_rate=100)

# Load scp_statements.csv for diagnostic aggregation
agg_df = pd.read_csv('../data/scp_statements.csv', index_col=0)
agg_df = agg_df[agg_df.diagnostic == 1]

def aggregate_diagnostic(y_dic):
    tmp = []
    for key in y_dic.keys():
        if key in agg_df.index:
            tmp.append(agg_df.loc[key].diagnostic_class)
    return list(set(tmp))


# Apply diagnostic superclass
Y['diagnostic_superclass'] = Y.scp_codes.apply(aggregate_diagnostic)

agg_df

Y_pd = pd.DataFrame(Y)
Y_pd
### Visualizing the Data
# Distribution of Male and Female ECGs
uniques, counts = np.unique(Y_pd.sex, return_counts=True)
plt.bar(uniques, counts)
for i in range(len(uniques)):
    plt.text(i, counts[i], str(counts[i]), ha='center', va='bottom')
    plt.text(i, i, str(i), ha='center', va='bottom', color = "white")
plt.title("Distribution of Sex")
diag_uniques, diag_counts = np.unique(Y_pd.diagnostic_superclass, return_counts=True)
temp = [str(i) for i in diag_uniques]
plt.bar(temp, diag_counts, width = 0.5)
for i in range(len(diag_uniques)):
    plt.text(i, diag_counts[i], str(diag_counts[i]), ha='center', va='bottom', size = 9, rotation = 60)

plt.xticks(rotation = 90)
plt.ylim(0, max(diag_counts) + max(diag_counts)*0.15)
plt.title("Distribution of Diagnostic Superclasses")
* NORM: Normal ECG
* MI: Myocardial Infarction
* STTC: ST/T Change
* CD: Conduction Disturbance
* HYP: Hypertrophy


To accurately predict the diagnostic superclass, much more data cleaning is required. 
* I will either need to trim the superclass list or spread out the concatenated data and one hot encode it. 
### Preprocessing
# Normalization
# Compute mean and standard deviation along axis 1 and 2
X_mean = np.mean(X, axis=(-1), keepdims=True)
X_std = np.std(X, axis=(-1), keepdims= True)

X_mean.shape
# Normalize data by subtracting mean and dividing by standard deviation
X_norm = (X - X_mean) / X_std
X_norm.shape

X_norm[(1,2,3),:,:]

# Split data into train and test
test_fold =10
X_train = X_norm[(Y.strat_fold != test_fold)]
y_train = Y[Y.strat_fold != test_fold].sex

X_test = X_norm[(Y.strat_fold == test_fold)]
y_test = Y[Y.strat_fold == test_fold].sex

y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

y_train = y_train.idxmax(axis = 1).to_numpy()
y_test = y_test.idxmax(axis = 1).to_numpy()

X_train.shape, X_test.shape, y_train.shape, y_test.shape

type(X_train)
# Evaluating data distribution
uniques, counts = np.unique(pd.DataFrame(y_train), return_counts=True)
plt.bar(uniques, counts)
for i in range(len(uniques)):
    plt.text(i, counts[i], str(counts[i]), ha='center', va='bottom')
    plt.text(i, i, str(i), ha='center', va='bottom', color = "white")
plt.title("Distribution of Sex")
# # Reshaping for ecg_sex_ecg_sex_ecg_sex_ecg_sex_ecg_sex_ecg_sex_model_8_8_8_8_8_8
# X_train = tf.reshape(X_train, (19601, 1000, 12,1))
# X_test = tf.reshape(X_test, (2198, 1000, 12,1))
# Model Creation
### Model 1
* Few Hidden layers
* Deafault 0.001 learning rate
* 2 layers of spatial analysis
* 1 Fully connected layer
* Relu and Sigmoid activations
tf.debugging.disable_traceback_filtering
# Random Seed
tf.random.set_seed(13)

# Creating the model

ecg_sex_model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters = 10, kernel_size = 7, strides = 2, padding = "valid", activation = 'relu'),
    tf.keras.layers.MaxPool1D(pool_size=2, strides = 1, padding = "valid"),
    tf.keras.layers.Conv1D(filters = 25, kernel_size = 4, strides = 1, padding = "valid", activation = 'relu'),
    tf.keras.layers.MaxPool1D(pool_size=2, strides = 1, padding = "valid"),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(25, activation= 'relu'),
    tf.keras.layers.Dense(1, activation= 'sigmoid')
])


# Compiling the model

ecg_sex_model.compile(loss = tf.keras.losses.binary_crossentropy,
                      optimizer = tf.keras.optimizers.Adam(learning_rate= 0.001),
                      metrics = ['accuracy'])

# Learning rate scheduler




# Fitting the model

history = ecg_sex_model.fit(X_train, y_train, epochs = 10, validation_data = (X_test, y_test))
### Model 2
* Modification of the number of neurons. 
* Adjusted learning rate slighlty
tf.debugging.disable_traceback_filtering
# Random Seed
tf.random.set_seed(13)

# Creating the model

ecg_sex_model_2 = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters = 32, kernel_size = 7, strides = 3, padding = "valid", activation='relu'),
    tf.keras.layers.MaxPool1D(pool_size=2, strides = 1, padding = "valid"),
    tf.keras.layers.Conv1D(filters = 64, kernel_size = 3, strides = 1, padding = "valid", activation = 'relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(15, activation= 'relu'),
    tf.keras.layers.Dense(1, activation= 'sigmoid')
])


# Compiling the model

ecg_sex_model_2.compile(loss = tf.keras.losses.binary_crossentropy,
                      optimizer = tf.keras.optimizers.Adam(learning_rate= 0.0015),
                      metrics = ['accuracy'])

# Learning rate scheduler




# Fitting the model

history_2 = ecg_sex_model_2.fit(X_train, y_train, epochs = 8, validation_data = (X_test, y_test))
### Model 3
* Additional spatial layer


**Key Issues at this time**
* The model appeared to be over fitting to the training data evidenced by the varying accuracies between training and testing
tf.debugging.disable_traceback_filtering
# Random Seed
tf.random.set_seed(13)

# Creating the model

ecg_sex_model_3 = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters = 32, kernel_size = 7, strides = 4, padding = "valid", activation='relu'),
    tf.keras.layers.MaxPool1D(pool_size=4, strides = 1, padding = "valid"),
    tf.keras.layers.Conv1D(filters = 64, kernel_size = 6, strides = 3, padding = "valid", activation = 'relu'),
    tf.keras.layers.MaxPool1D(pool_size=4, strides = 1, padding = "valid"),
    tf.keras.layers.Conv1D(filters = 96, kernel_size = 4, strides = 2, padding = "valid", activation = 'relu'),
    tf.keras.layers.MaxPool1D(pool_size=2, strides = 1, padding = "valid"),
    tf.keras.layers.Dense(25, activation= 'relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1, activation= 'sigmoid')
])


# Compiling the model

ecg_sex_model_3.compile(loss = tf.keras.losses.binary_crossentropy,
                      optimizer = tf.keras.optimizers.Adam(learning_rate= 0.0015),
                      metrics = ['accuracy'])

# Learning rate scheduler




# Fitting the model

history_3 = ecg_sex_model_3.fit(X_train, y_train, epochs = 10, validation_data = (X_test, y_test))
### Model 4
* Implemented learning rate scheduler callback
* Implemented early stop callback
* Additional spatial layer
* Weaved in batch normalization
* Implemented weight droppout
tf.debugging.disable_traceback_filtering
# Random Seed
tf.random.set_seed(13)

# Creating the model

ecg_sex_model_4 = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters = 32, kernel_size = 7, strides = 4, padding = "valid", activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool1D(pool_size=4, strides = 1, padding = "valid"),

    tf.keras.layers.Conv1D(filters = 48, kernel_size = 6, strides = 3, padding = "valid", activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool1D(pool_size=3, strides = 1, padding = "valid"),
    tf.keras.layers.Dropout(0.2, seed = 13),

    tf.keras.layers.Conv1D(filters = 64, kernel_size = 4, strides = 2, padding = "valid", activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool1D(pool_size=2, strides = 1, padding = "valid"),

    tf.keras.layers.Conv1D(filters = 96, kernel_size = 3, strides = 1, padding = "valid", activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool1D(pool_size=2, strides = 1, padding = "valid"),
    tf.keras.layers.Flatten(),


    tf.keras.layers.Dense(64, activation= 'relu'),
    tf.keras.layers.Dropout(0.2, seed = 13),
    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(1, activation= 'sigmoid')
])


# Compiling the model

ecg_sex_model_4.compile(loss = tf.keras.losses.binary_crossentropy,
                      optimizer = tf.keras.optimizers.Adam(learning_rate= 0.001),
                      metrics = ['accuracy'])

# Learning rate scheduler
early_stopper = tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy', min_delta= 0.01,patience = 4, baseline= 0.75)



# Fitting the model

history_4 = ecg_sex_model_4.fit(X_train, y_train, epochs = 12, validation_data = (X_test, y_test), callbacks = early_stopper)

### Model 5
* 5 spatial layers each with their own batch normalization
* Modified learning rate
* Implemented Lasso L1 regularizer. 
* Early stopper in place
tf.debugging.disable_traceback_filtering
# Random Seed
tf.random.set_seed(13)
tf.keras.backend.set_image_data_format('channels_first')


# Creating the model

ecg_sex_model_5 = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters = 32, kernel_size = 8, strides = 4, padding = "valid", activation='relu', input_shape= (1000, 12)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool1D(pool_size=4, strides = 1, padding = "valid"),

    tf.keras.layers.Conv1D(filters = 48, kernel_size = 7, strides = 3, padding = "valid", activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool1D(pool_size=2, strides = 1, padding = "valid"),
    tf.keras.layers.Dropout(0.2, seed = 13),

    tf.keras.layers.Conv1D(filters = 64, kernel_size = 5, strides = 2, padding = "valid", activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool1D(pool_size=2, strides = 1, padding = "valid"),

    tf.keras.layers.Conv1D(filters = 96, kernel_size = 4, strides = 1, padding = "valid", activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool1D(pool_size=2, strides = 1, padding = "valid"),

    tf.keras.layers.Conv1D(filters = 128, kernel_size = 3, strides = 1, padding = "valid", activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool1D(pool_size=2, strides = 1, padding = "valid"),


    tf.keras.layers.Dense(64, activation= 'relu', kernel_regularizer='l1'),
    tf.keras.layers.Dropout(0.2, seed = 13),
    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(1, activation= 'sigmoid')
])


# Compiling the model

ecg_sex_model_5.compile(loss = tf.keras.losses.binary_crossentropy,
                      optimizer = tf.keras.optimizers.Adam(learning_rate= 0.0009),
                      metrics = ['accuracy'])

# Learning rate scheduler
early_stopper = tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy', min_delta= 0.01,patience = 4, baseline= 0.75)
#lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lambda epoch: (0.0008) * 10**(epoch/20))



# Fitting the model

history_5 = ecg_sex_model_5.fit(X_train, y_train, epochs = 12, validation_data = (X_test, y_test), callbacks = (early_stopper))
### Model 6
* Wanted to see if model's performance was proportional to training time
* Doubled the epochs and disabled the early stopper


**Take aways**
* The model's training accuracy consistently grew with epochs
* The model's testing accuracy fluctuated and plateaued around 0.8
* The number of epochs has little effect on testing accuracy when the model overfits.
tf.debugging.disable_traceback_filtering
# Random Seed
tf.random.set_seed(13)

# Creating the model

ecg_sex_model_6 = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters = 32, kernel_size = 8, strides = 4, padding = "valid", activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool1D(pool_size=4, strides = 1, padding = "valid"),

    tf.keras.layers.Conv1D(filters = 64, kernel_size = 7, strides = 3, padding = "valid", activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool1D(pool_size=2, strides = 1, padding = "valid"),
    tf.keras.layers.Dropout(0.2, seed = 13),

    tf.keras.layers.Conv1D(filters = 96, kernel_size = 5, strides = 2, padding = "valid", activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool1D(pool_size=2, strides = 1, padding = "valid"),

    tf.keras.layers.Conv1D(filters = 128, kernel_size = 4, strides = 1, padding = "valid", activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool1D(pool_size=2, strides = 1, padding = "valid"),

    tf.keras.layers.Conv1D(filters = 160, kernel_size = 3, strides = 1, padding = "valid", activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool1D(pool_size=2, strides = 1, padding = "valid"),
    tf.keras.layers.Flatten(),


    tf.keras.layers.Dense(64, activation= 'relu', kernel_regularizer='l1'),
    tf.keras.layers.Dropout(0.2, seed = 13),

    tf.keras.layers.Dense(1, activation= 'sigmoid')
])


# Compiling the model

ecg_sex_model_6.compile(loss = tf.keras.losses.binary_crossentropy,
                      optimizer = tf.keras.optimizers.Adam(learning_rate= 0.0009),
                      metrics = ['accuracy'])

# Learning rate scheduler
early_stopper = tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy', min_delta= 0.01,patience = 4, baseline= 0.75)
#lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lambda epoch: (0.0008) * 10**(epoch/20))



# Fitting the model

history_6 = ecg_sex_model_6.fit(X_train, y_train, epochs = 20, validation_data = (X_test, y_test))

### Model 7
* 8 spatial layers
* 1 temporal layer
* 2 Fully connected layer
* https://www.ahajournals.org/doi/full/10.1161/CIRCEP.119.007284
tf.debugging.disable_traceback_filtering
tf.config.list_physical_devices('GPU')
# Random Seed
tf.random.set_seed(13)

# Creating the model

ecg_sex_model_7 = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters = 16, kernel_size = 7, strides = 1, padding = "valid", activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool1D(pool_size=2, strides = 1, padding = "valid"),

    tf.keras.layers.Conv1D(filters = 16, kernel_size = 5, strides = 1, padding = "valid", activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool1D(pool_size=5, strides = 1, padding = "valid"),
    tf.keras.layers.Dropout(0.2, seed = 13),

    tf.keras.layers.Conv1D(filters = 32, kernel_size = 5, strides = 1, padding = "valid", activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool1D(pool_size=2, strides = 1, padding = "valid"),

    tf.keras.layers.Conv1D(filters = 32, kernel_size = 5, strides = 1, padding = "valid", activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool1D(pool_size=4, strides = 1, padding = "valid"),

    tf.keras.layers.Conv1D(filters = 64, kernel_size = 3, strides = 1, padding = "valid", activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool1D(pool_size=2, strides = 1, padding = "valid"),

    tf.keras.layers.Conv1D(filters = 64, kernel_size = 3, strides = 1, padding = "valid", activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool1D(pool_size=2, strides = 1, padding = "valid"),

    tf.keras.layers.Conv1D(filters = 64, kernel_size = 3, strides = 1, padding = "valid", activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool1D(pool_size=2, strides = 1, padding = "valid"),

    tf.keras.layers.Conv1D(filters = 64, kernel_size = 3, strides = 1, padding = "valid", activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool1D(pool_size=2, strides = 1, padding = "valid"),

    tf.keras.layers.Conv1D(filters = 128, kernel_size = 12, strides = 1, padding = "valid", activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool1D(pool_size=2, strides = 1, padding = "valid"),

    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(128, activation= 'relu', kernel_regularizer='l1'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2, seed = 13),


    tf.keras.layers.Dense(64, activation= 'relu', kernel_regularizer='l1'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2, seed = 13),

    tf.keras.layers.Dense(1, activation= 'sigmoid')
])


# Compiling the model

ecg_sex_model_7.compile(loss = tf.keras.losses.binary_crossentropy,
                      optimizer = tf.keras.optimizers.Adam(learning_rate= .0001),
                      metrics = ['accuracy'])

# Learning rate scheduler
early_stopper = tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy', min_delta= 0.01,patience = 5)
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lambda epoch: (0.0008) * 10**(epoch/20))



# Fitting the model

history_7 = ecg_sex_model_7.fit(X_train, y_train, epochs = 25, validation_data = (X_test, y_test))
import itertools
from sklearn.metrics import confusion_matrix
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
# Note: The following confusion matrix code is a remix of Scikit-Learn's 
# plot_confusion_matrix function - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.plot_confusion_matrix.html
# and Made with ML's introductory notebook - https://github.com/GokuMohandas/MadeWithML/blob/main/notebooks/08_Neural_Networks.ipynb 
import itertools

figsize = (10, 10)

def make_confusion_matrix(X_test, y_test, classes, model, figsize=(18,18), text_size = 15):
    # Create the confusion matrix
    y_prob = model.predict(X_test)
    y_pred =np.squeeze(np.where(y_prob >= 0.5, 1, 0))
    cm = confusion_matrix(y_test, tf.round(y_pred))
    cm_norm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis] # normalize it
    n_classes = cm.shape[0]

    # Let's prettify it
    fig, ax = plt.subplots(figsize=figsize)
    # Create a matrix plot
    cax = ax.matshow(cm, cmap=plt.cm.Blues) # https://matplotlib.org/3.2.0/api/_as_gen/matplotlib.axes.Axes.matshow.html
    fig.colorbar(cax)

    # Create classes
    if classes:
        labels = classes
    else:
        labels = np.arange(cm.shape[0])

    # Label the axes
    ax.set(title=(str(model.name) + " for Confusion Matrix"),
        xlabel="Predicted label",
        ylabel="True label",
        xticks=np.arange(n_classes),
        yticks=np.arange(n_classes),
        xticklabels=labels,
        yticklabels=labels)

    # Set x-axis labels to bottom
    ax.xaxis.set_label_position("bottom")
    ax.xaxis.tick_bottom()

    # Adjust label size
    ax.xaxis.label.set_size(20)
    ax.yaxis.label.set_size(20)
    ax.title.set_size(20)

    # Set threshold for different colors
    threshold = (cm.max() + cm.min()) / 2.

    # Plot the text on each cell
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, f"{cm[i, j]} ({cm_norm[i, j]*100:.1f}%)",
                horizontalalignment="center",
                color="white" if cm[i, j] > threshold else "black",
                size=10)
### Visualization of data
y_prob = ecg_sex_model_6.predict(X_test)
y_pred =np.squeeze(np.where(y_prob >= 0.5, 1, 0))


class_name = ['Male', 'Female']
make_confusion_matrix(X_test, y_test, model = ecg_sex_model_6, classes = class_name)


# Visualizing the ECGs with model interpretation
* External resources:
    * https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2760216/
from random import choice

def visualize(y_test, X_test, model):
    class_names  = ['Male', 'Female']
    y_prob = model.predict(X_test)
    y_pred =np.squeeze(np.where(y_prob >= 0.5, 1, 0))
    for i in range(4):
        ind = choice(range(1, len(X_test)))
        plot_title = ("Actual: " + class_names[y_test[ind]] + "    Predicted: " + class_names[y_pred[ind]])
        ecg_plot.plot_1(tf.transpose(X, (0,2,1))[ind][4], sample_rate=100, title = plot_title + "   aVL focus")

visualize(y_test, X_test, ecg_sex_model_6)
from random import choice

def visualize(y_test, X_test, model):
    class_names  = ['Male', 'Female']
    y_prob = model.predict(X_test)
    y_pred =np.squeeze(np.where(y_prob >= 0.5, 1, 0))
    for i in range(4):
        ind = choice(range(1, len(X_test)))
        plot_title = ("Actual: " + class_names[y_test[ind]] + "    Predicted: " + class_names[y_pred[ind]])
        ecg_plot.plot_1(tf.transpose(X, (0,2,1))[ind][4], sample_rate=100, title = plot_title + "   aVL focus")

visualize(y_test, X_test, ecg_sex_model_6)

### Conclusions
* Attempt to train on the 500hz data set
* Use a cloud computing system to train larger models such as Google Collab. 
* Fine tune an existing high performance model such as ResNet. 